# Extract a few individuals as examples


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import subprocess
import sys
import pandas as pd
import numpy as np
import geopandas as gpd

def get_repo_root():
    """Get the root directory of the repo."""
    dir_in_repo = os.path.dirname(os.path.abspath('__file__')) # os.getcwd()
    return subprocess.check_output('git rev-parse --show-toplevel'.split(),
                                   cwd=dir_in_repo,
                                   universal_newlines=True).rstrip()
sys.path.append(get_repo_root())
ROOT_dir = get_repo_root()

import lib.mscthesis as mscthesis
import lib.netherlands as netherlands
import lib.genericvalidation as genericvalidation

## 1 Load users' visits stats to select 4 individuals

In [2]:
df = pd.read_csv(ROOT_dir + '/dbs/netherlands/visits/visits_3_stats.csv')
df_users = df.groupby('userid')['pkt'].agg(['mean', 'min', 'max'])

## 2 Create individual OD with their synthesized visits

In [3]:
users = [763009, 767248, 736794]
df_visits = pd.read_csv(ROOT_dir + '/dbs/netherlands/visits/visits_3.csv')
df_visits = df_visits.loc[df_visits.userid.isin(users), :]

In [4]:
df_visits.userid.nunique()

2

In [5]:
model_odms = []
## Get spatial zones
gs = netherlands.GroundTruthLoader()
gs.load_zones()
zones = gs.zones
for user, ele in df_visits.groupby('userid'):
    od = genericvalidation.visits_to_odm(ele.set_index('userid'), zones)
    od.name = f'user_{user}'
    model_odms.append(od)

Convering visits to zone CRS
Aligning region-visits to zones...
removed 0 region-visits due to missing zone geom
Aligning point-visits to zones...
removed 0 point-visits due to missing zone geom
622 visits left after alignment
Creating odm...
Convering visits to zone CRS
Aligning region-visits to zones...
removed 0 region-visits due to missing zone geom
Aligning point-visits to zones...
removed 1 point-visits due to missing zone geom
612 visits left after alignment
Creating odm...


In [6]:
df_od = pd.concat(model_odms, axis=1).reset_index()
df_od = df_od.rename(columns={'level_0': 'ozone', 'level_1': 'dzone'})
df_od.columns

Index(['ozone', 'dzone', 'user_763009', 'user_767248'], dtype='object')

In [7]:
df_od = df_od.loc[(df_od['user_763009'] != 0) | (df_od['user_767248'] != 0), :]
df_od.to_csv(ROOT_dir + '/dbs/netherlands_visits_3_examples.csv', index=False)